In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sys import path
%matplotlib inline

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')
%cd /content/drive/MyDrive/PHI/ToF_ML/src

/content/drive/MyDrive/PHI/ToF_ML/src


In [4]:
from data_transformation import get_frags
spots = get_frags()['FragmentMass']

In [5]:
def C(channels, spec_bin_size, start_time,  mass_over_time, mass_offset):
    return ((np.array(channels) * .001 * spec_bin_size + start_time) * mass_over_time + mass_offset)**2

In [6]:
df_read = pd.read_csv('../data/ssl_csvs_converted_to_df.csv')
from ast import literal_eval
df_read['intensities'] = df_read['intensities'].apply(literal_eval)
df_read['channels'] = df_read['channels'].apply(literal_eval)

In [7]:
df_read.head()

,file_name,technique,StartFlightTime,MassOverTime,MassOffset,SpecBinSize,intensities,channels
0,C0059-V1.cas,-TofSIMS,0.0,1.69097,0.70950,0.128,"[10106.0, 32018.0, 4303.0, 816.0, 44497.0, 346...","[12729.6134, 13388.07292, 14021.38724, 14625.2..."
1,C0060-V1.cas,-TofSIMS,0.0,1.68665,0.70710,0.128,"[37242.0, 67973.0, 112.0, 5361.0, 4489.0, 2193...","[12773.45818, 13433.49441, 13495.05187, 14068...."
2,C0061-U1.cas,+TofSIMS,0.0,1.68482,0.70859,0.128,"[483.0, 782.0, 1374.0, 2985.0, 152.0, 302.0, 3...","[12779.34033, 13440.22492, 14076.03163, 14689...."
3,C0061-V1.cas,-TofSIMS,0.0,1.69038,0.70903,0.128,"[2675.0, 1381.0, 674.0, 196.0, 93.0, 50539.0, ...","[1381.98178, 1400.517, 1412.53457, 1440.11759,..."
4,C0062-U1.cas,+TofSIMS,0.0,1.68314,0.70792,0.128,"[565.0, 835.0, 1243.0, 2532.0, 99.0, 220.0, 14...","[12795.06881, 13456.57005, 14092.96684, 14707...."


In [8]:
from data_transformation import mass_formula
def generate_calibrated_data(data, numba=False):
    '''
    Applies mass_formula to every row in dataset to allow
    calibrated graphs to be generated.
    '''
    new_data = data.copy()
    masses = []
    for row in new_data.itertuples():
        spec = row.SpecBinSize
        m_over_t = row.MassOverTime
        m_offset = row.MassOffset
        time = row.StartFlightTime
        if not numba:
            masses.append(mass_formula(np.array(row.channels), spec, time,
                          m_over_t, m_offset))
        else:
            masses.append(numba_mass_formula(np.array(row.channels), spec,
                          time, m_over_t, m_offset))
    new_data['masses'] = masses
    return new_data

In [9]:
from data_transformation import get_fragment_stats, generate_calibrated_data
df_read = generate_calibrated_data(df_read[df_read['channels'].apply(len)> 0])
df_read.reset_index(inplace=True)

In [10]:
df_read.head()

,index,file_name,technique,StartFlightTime,MassOverTime,MassOffset,SpecBinSize,intensities,channels,masses
0,0,C0059-V1.cas,-TofSIMS,0.0,1.69097,0.70950,0.128,"[10106.0, 32018.0, 4303.0, 816.0, 44497.0, 346...","[12729.6134, 13388.07292, 14021.38724, 14625.2...","[12.004495885110552, 13.012398275521816, 14.02..."
1,1,C0060-V1.cas,-TofSIMS,0.0,1.68665,0.70710,0.128,"[37242.0, 67973.0, 112.0, 5361.0, 4489.0, 2193...","[12773.45818, 13433.49441, 13495.05187, 14068....","[12.004681147019333, 13.01242014410188, 13.108..."
2,2,C0061-U1.cas,+TofSIMS,0.0,1.68482,0.70859,0.128,"[483.0, 782.0, 1374.0, 2985.0, 152.0, 302.0, 3...","[12779.34033, 13440.22492, 14076.03163, 14689....","[12.003062954788383, 13.010939902638805, 14.01..."
3,3,C0061-V1.cas,-TofSIMS,0.0,1.69038,0.70903,0.128,"[2675.0, 1381.0, 674.0, 196.0, 93.0, 50539.0, ...","[1381.98178, 1400.517, 1412.53457, 1440.11759,...","[1.016159799036636, 1.024261311539043, 1.02953..."
4,4,C0062-U1.cas,+TofSIMS,0.0,1.68314,0.70792,0.128,"[565.0, 835.0, 1243.0, 2532.0, 99.0, 220.0, 14...","[12795.06881, 13456.57005, 14092.96684, 14707....","[12.002858579352937, 13.010660285835183, 14.01..."


In [11]:
df = get_fragment_stats(df_read, prop_name='proportions_peaks_identified')

In [12]:
df.to_csv('../data/new_1900_without_peakless_spectra.csv', index=False)

In [13]:
a = df['index']
df.drop('index', axis=1, inplace=True)
df['index'] = a

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1911 entries, 0 to 1910
Data columns (total 17 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   file_name                              1911 non-null   object 
 1   technique                              1911 non-null   object 
 2   StartFlightTime                        1911 non-null   float64
 3   MassOverTime                           1911 non-null   float64
 4   MassOffset                             1911 non-null   float64
 5   SpecBinSize                            1911 non-null   float64
 6   intensities                            1911 non-null   object 
 7   channels                               1911 non-null   object 
 8   masses                                 1911 non-null   object 
 9   avg_dist_frags_low                     1911 non-null   float64
 10  avg_dist_frags_high                    1911 non-null   float64
 11  adju

In [15]:
df.head()

,file_name,technique,StartFlightTime,MassOverTime,MassOffset,SpecBinSize,intensities,channels,masses,avg_dist_frags_low,avg_dist_frags_high,adjusted_proportions_peaks_identified,proportions_peaks_identified,diff,prop_diff_in_low,calibration,index
0,C0059-V1.cas,-TofSIMS,0.0,1.69097,0.70950,0.128,"[10106.0, 32018.0, 4303.0, 816.0, 44497.0, 346...","[12729.6134, 13388.07292, 14021.38724, 14625.2...","[12.004495885110552, 13.012398275521816, 14.02...",0.001419,0.003314,0.307692,0.294737,0.001895,1.335205,0,0
1,C0060-V1.cas,-TofSIMS,0.0,1.68665,0.70710,0.128,"[37242.0, 67973.0, 112.0, 5361.0, 4489.0, 2193...","[12773.45818, 13433.49441, 13495.05187, 14068....","[12.004681147019333, 13.01242014410188, 13.108...",0.001261,0.002194,0.409091,0.393939,0.000933,0.740399,0,1
2,C0061-U1.cas,+TofSIMS,0.0,1.68482,0.70859,0.128,"[483.0, 782.0, 1374.0, 2985.0, 152.0, 302.0, 3...","[12779.34033, 13440.22492, 14076.03163, 14689....","[12.003062954788383, 13.010939902638805, 14.01...",0.001245,0.002281,0.428571,0.416058,0.001036,0.831804,0,2
3,C0061-V1.cas,-TofSIMS,0.0,1.69038,0.70903,0.128,"[2675.0, 1381.0, 674.0, 196.0, 93.0, 50539.0, ...","[1381.98178, 1400.517, 1412.53457, 1440.11759,...","[1.016159799036636, 1.024261311539043, 1.02953...",0.001129,0.002034,0.305155,0.241830,0.000905,0.801556,0,3
4,C0062-U1.cas,+TofSIMS,0.0,1.68314,0.70792,0.128,"[565.0, 835.0, 1243.0, 2532.0, 99.0, 220.0, 14...","[12795.06881, 13456.57005, 14092.96684, 14707....","[12.002858579352937, 13.010660285835183, 14.01...",0.001288,0.002298,0.427509,0.424354,0.001011,0.784703,0,4


In [16]:
def recalibrate(row, spots, slope_amt=0, offset_amt=0, num_peaks=False):
    '''
    Function which quickly asseses the calibration of a spectrum.
    '''
    slope = row.MassOverTime + slope_amt *  row.MassOverTime
    offset = row.MassOffset + offset_amt * row.MassOffset
    peaks = []
    peaks = mass_formula(np.array(row.channels), row.SpecBinSize, row.StartFlightTime, slope,
                   offset)
    masses, frags1, dist1, masses2, frags2, dist2 = get_frags_dists(peaks,
                                                                    spots,
                                                                    [.003,.007])
    prop = len(masses) / len(peaks[peaks<336])
    low_dist = 0
    high_dist = 0
    if len(dist1) > 0:
        low_dist = np.mean(dist1)
    if len(dist2) > 0:
        high_dist = np.mean(dist2)
    if num_peaks:
        return prop, low_dist, high_dist, len(masses)
    else:
        return prop, low_dist, high_dist

In [17]:
from data_transformation import get_frags_dists

In [18]:
def get_best_offset(spectrum, slope_range, offset_range, prev=0,
                    offsets=30, slopes=20, first=True, frags=None):
    '''
    Find best amount of slope/offset to add/subtract to slope/offset value
    to achieve the optimal calibration for spectrum. Calibration is measured
    using mass fragments. A spectrum which more matches to known masses is more
    calibrated than one with fewer. A spectrum whose matches are very close to
    known mass is more calibrated than one that is further away.

    Arguments -------
    spectrum: row from dataframe containing information on a spectrum
    slope_range: data structure containing min, max slope to try, slope erros
                 are typically smaller than offset errors.
    offset_range: data structure containing min, max offset agumentation
                       to try. This method shrinks the range iteratively until
                       the best offset is achieved.
    spots: a list of mass fragments
    '''
    if first:
        print('optimizing ' + spectrum.file_name)
        print('initial proportion: ' + str(spectrum.proportions_peaks_identified))
    if not isinstance(frags, pd.Series):
        frags = get_frags()
    proportions = [0]
    low_distances = []
    high_distances = []
    best_prop = 0
    best_offset = spectrum.MassOffset
    best_slope = spectrum[2]
    best_ld = 1
    best_hd = 1
    ys = [] 
    i = 0
    mults = [1, -1]
    slope_space = np.linspace(slope_range[0], slope_range[1], slopes)
    while 1:
        slope = slope_space[i]
        for slope_mult in mults:
            slope_val = slope * slope_mult
            props = []
            low_dists = []
            high_dists = []
            y = []
            space = np.linspace(offset_range[0], offset_range[1], offsets)
            j = 0
            changed = False
            while 1:
                offset = space[j]
                for mult in mults:
                    improved = False
                    val = mult * offset
                    y.append(val)
                    prop, low, high = recalibrate(spectrum, frags, slope_val, val)
                    if prop > best_prop:
                        improved=True
                    elif (prop == best_prop and best_hd - high > 0 and
                          best_ld - low > 0):
                        improved = True
                    props.append(prop)
                    low_dists.append(low)
                    high_dists.append(high)
                    if improved:
                        best_prop = prop 
                        best_offset = val
                        best_slope = slope_val
                        best_ld = low
                        best_hd = high
                        edge = (np.where(space == mult * best_offset)[0][0] + .1)/ len(space)
                        edge = 2 * abs(.5 - edge)
                        slope_edge = np.where(slope_space == slope_mult * best_slope)[0][0]
                        slope_edge = 2 * abs(.5 - (slope_edge + .1) / len(slope_space))
                        changed = True            
                j += 1
                if j >= len(space):
                    break
            if changed:
                ys = y 
                proportions = props 
                low_distances = low_dists
                high_distances = high_dists
        i += 1
        if i >= len(slope_space):
            break
    print(best_prop)
    if best_prop > prev:
            a = best_offset - (.5/edge) * best_offset
            b = best_offset + (.5/edge) * best_offset
            c = best_slope + (.5 / slope_edge) * best_slope
            d = best_slope - (.5 / slope_edge) * best_slope
            p, ld, hd, yss, bp, bo, bs = get_best_offset(spectrum,
                                                            slope_range=[c,d],
                                                            offset_range=[a, b],
                                                            prev=best_prop,
                                                            offsets=20,
                                                            slopes=5, 
                                                            first=False, 
                                                            frags=frags)
            if bp >= best_prop:
                proportions = p
                low_distances = ld
                high_distances = hd
                ys = yss
                best_prop = bp
                best_offset = bo
                best_slope = bs
    return (proportions, low_distances, high_distances, ys, best_prop,
            best_offset, best_slope)

In [19]:
offsets = []
slopes = []
files = []
indices = []
i = 0
for row in df.loc[956:1434].itertuples():
    print(i)
    a, b, c, d, p, o, s = get_best_offset(row, [.001, .0000001],
                                          [.001, .000001], offsets=20,
                                          slopes=20,
                                          prev=0,
                                          frags=spots)
    offsets.append(o)
    slopes.append(s)
    files.append(row.file_name)
    indices.append(row.index)
    i += 1

0.6523297491039427
452
optimizing B1575-V1.cas
initial proportion: 0.5729166666666666
0.8020833333333334
0.796875
453
optimizing B1576-U1.cas
initial proportion: 0.5202020202020202
0.7575757575757576
0.7474747474747475
454
optimizing B1576-V1.cas
initial proportion: 0.5185185185185185
0.7901234567901234
0.7962962962962963
0.7839506172839507
455
optimizing B1577-U1.cas
initial proportion: 0.20616570327552985
0.35645472061657035
0.3603082851637765
0.3603082851637765
456
optimizing B1578-U1.cas
initial proportion: 0.3729281767955801
0.5359116022099447
0.5441988950276243
0.5386740331491713
457
optimizing B1577-V1.cas
initial proportion: 0.5352112676056338
0.8028169014084507
0.7981220657276995
458
optimizing B1578-V1.cas
initial proportion: 0.5175879396984925
0.7738693467336684
0.7788944723618091
0.7738693467336684
459
optimizing B1579-U1.cas
initial proportion: 0.38636363636363635
0.6071428571428571
0.6136363636363636
0.6006493506493507
460
optimizing B1579-V1.cas
initial proportion: 0.560

In [20]:
pd.DataFrame({'offsets':offsets, 'slopes':slopes, 'names':files, 'indices':indices}).to_csv('../data/updated_calibration_new_1900_first_956_478.csv', index=False)